## Info

This is a slightly modified version of 

`Build Data Multi-Asset ETFs.ipynb`

In [1]:
import pandas as pd
import numpy as np

import yfinance as yf

/Users/markhendricks/Projects/drw-dataanalysis-2024/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
TICKS = [
    'SPY',
    'EFA',
    'EEM',
    'PSP',
    'QAI',
    'HYG',
    'DBC',
    'IYR',
    'IEF',
    'BWX',
    'TIP',
    'SHV',
]

FLDS = ['shortName','quoteType','currency','volume','totalAssets','longBusinessSummary']

In [3]:
info = pd.DataFrame(index=TICKS,columns=FLDS)
info.index.name = 'ticker'
for tick in info.index:
    temp = yf.Ticker(tick).get_info()

    for fld in FLDS:
        if fld in temp.keys():
            info.loc[tick,fld] = temp[fld]

In [4]:
info

,shortName,quoteType,currency,volume,totalAssets,longBusinessSummary
ticker,,,,,,
SPY,SPDR S&P 500,ETF,USD,31915489,533350318080,The Trust seeks to achieve its investment obje...
EFA,iShares MSCI EAFE ETF,ETF,USD,14936082,54361296896,The fund generally will invest at least 80% of...
EEM,iShares MSCI Emerging Index Fun,ETF,USD,18737887,18598756352,The fund generally will invest at least 80% of...
PSP,Invesco Global Listed Private E,ETF,USD,7955,248697808,The fund generally will invest at least 90% of...
QAI,IQ Hedge MultiIQ Hedge Multi-St,ETF,USD,48268,560306880,"The fund is a ""fund of funds"" which means it i..."
HYG,iShares iBoxx $ High Yield Corp,ETF,USD,51364193,16733312000,The underlying index is a rules-based index co...
DBC,Invesco DB Commodity Index Trac,ETF,USD,891311,1784920832,The fund pursues its investment objective by i...
IYR,iShares U.S. Real Estate ETF,ETF,USD,3489431,2901542144,The fund seeks to track the investment results...
IEF,iShares 7-10 Year Treasury Bond,ETF,USD,5404923,29142607872,The underlying index measures the performance ...


In [5]:
STARTDATE = '2009-03-31'
ENDDATE = '2024-05-31'

tickers = list(info.index.values)
df = yf.download(tickers, start=STARTDATE, end=ENDDATE)['Adj Close']

[*********************100%%**********************]  12 of 12 completed


In [6]:
prices = df.resample('M').last()

rets = prices.pct_change().dropna()
# reorder columns to match info tab, not alphabetical
rets = rets[info.index]

# change to excess returns, in excess of short-term treasury
retsx = rets.subtract(rets['SHV'], axis=0)
retsx = retsx.drop(columns=['SHV'])

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_11026/2262798614.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  prices = df.resample('M').last()


In [7]:
rets

Ticker,SPY,EFA,EEM,PSP,QAI,HYG,DBC,IYR,IEF,BWX,TIP,SHV
Date,,,,,,,,,,,,
2009-04-30,0.099346,0.115190,0.155582,0.230203,0.022883,0.138460,-0.001000,0.296151,-0.027453,0.008993,-0.017951,0.000553
2009-05-31,0.058454,0.131918,0.159400,0.053892,0.027865,0.028555,0.162663,0.022728,-0.020773,0.053672,0.019966,-0.000472
2009-06-30,-0.000655,-0.014049,-0.022495,0.045449,-0.003436,0.033517,-0.026259,-0.024863,-0.005571,0.005148,0.001981,0.000599
2009-07-31,0.074606,0.100415,0.110146,0.143247,0.015326,0.069190,0.018568,0.105799,0.008317,0.031284,0.000879,-0.000027
2009-08-31,0.036940,0.045030,-0.013136,0.033413,-0.004151,-0.016969,-0.040365,0.131939,0.007635,0.007628,0.008413,0.000436
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-31,0.015926,-0.004512,-0.045262,-0.006270,-0.003987,0.001163,0.013158,-0.050979,0.000726,-0.030630,0.003349,0.003995
2024-02-29,0.052187,0.029863,0.041678,0.041903,0.015677,0.003035,-0.015226,0.021326,-0.020826,-0.010788,-0.010477,0.003933
2024-03-31,0.032702,0.033786,0.027257,0.032782,0.014450,0.010898,0.044566,0.018538,0.007332,0.003882,0.006466,0.004161


In [8]:
TICKS_PORT = ['PSP','QAI','IYR','IEF']
port = (rets[TICKS_PORT].mean(axis=1)).to_frame().rename(columns={0:'portfolio'})

In [9]:
with pd.ExcelWriter('../data/multi_asset_etf_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name= 'info')
    #prices.to_excel(writer, sheet_name= 'prices')
    rets.to_excel(writer, sheet_name='total returns')
    port.to_excel(writer, sheet_name='portfolio returns')
    #retsx.to_excel(writer, sheet_name='excess returns')